# Salzburg ICU Data Preprocessing

In [1]:
import pandas as pd
from collections import Counter

import dask.array as da 
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import numpy as np

from multiprocessing import Pool

from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import math

from tqdm import tqdm
import time

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
case_data = pd.read_csv('../icu_data/salzburg/cases.csv.gz', compression = 'gzip')
d_reference = pd.read_csv('../icu_data/salzburg/d_references.csv.gz', compression = 'gzip')
data_float = pd.read_csv('../icu_data/salzburg/data_float_h.csv.gz', compression = 'gzip')
data_range = pd.read_csv('../icu_data/salzburg/data_range.csv.gz', compression = 'gzip')
data_ref = pd.read_csv('../icu_data/salzburg/data_ref.csv.gz', compression = 'gzip')
lab_data = pd.read_csv('../icu_data/salzburg/laboratory.csv.gz', compression = 'gzip')
medication_data = pd.read_csv('../icu_data/salzburg/medication.csv.gz', compression = 'gzip')
unit_log_data = pd.read_csv('../icu_data/salzburg/unitlog.csv.gz', compression = 'gzip')

In [4]:
case_data

,CaseID,PatientID,AdmissionYear,TimeOfStay,ICUOffset,saps3,HospitalDischargeType,HospitalDischargeDay,HospitalStayDays,DischargeState,...,AdmissionUrgency,AdmissionFormHasSepsis,SurgicalAdmissionType,OrbisDataAvailable,HeartSurgeryAdditionalData,HeartSurgeryCPBTime,HeartSurgeryCrossClampTime,HeartSurgeryBeginOffset,HeartSurgeryEndOffset,OffsetAfterFirstAdmission
0,100023,163229,2016,17100,0,58,3130.0,2.0,2.0,2202,...,3137,738,3127,740,738,NaN,NaN,NaN,NaN,0
1,100027,541172,2016,273600,0,46,3129.0,3.0,6.0,2202,...,3137,738,3127,740,738,NaN,NaN,NaN,NaN,0
2,100066,151137,2017,475200,0,43,3131.0,6.0,6.0,2202,...,3137,738,3127,740,738,NaN,NaN,NaN,NaN,0
3,100097,947681,2016,79140,0,21,3129.0,4.0,8.0,2202,...,3138,738,3126,740,738,NaN,NaN,NaN,NaN,0
4,100130,717997,2019,64800,0,60,3131.0,44.0,44.0,2202,...,3137,740,3127,740,738,NaN,NaN,NaN,NaN,69481020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27345,999924,787696,2015,435600,16740,43,3131.0,7.0,12.0,2202,...,3138,738,3126,740,738,NaN,NaN,NaN,NaN,0
27346,999926,464697,2017,266940,31440,49,3129.0,11.0,16.0,2202,...,3138,738,3126,740,738,NaN,NaN,NaN,NaN,0
27347,999934,324646,2019,201600,0,34,3129.0,9.0,9.0,2212,...,3136,738,3124,740,738,NaN,NaN,NaN,NaN,0
27348,999947,850335,2017,507600,22500,77,3129.0,24.0,25.0,2202,...,3137,738,3125,740,740,189.0,143.0,3780.0,22500.0,99783540


In [5]:
d_reference

,ReferenceGlobalID,ReferenceValue,ReferenceName,ReferenceDescription,ReferenceUnit,ReferenceOrder,ReferenceType,Data,LOINC_code,LOINC_short,LOINC_long
0,2,INIC,HospitalUnit,9 bed intermediate care ward,NaN,0,0.0,NaN,NaN,NaN,NaN
1,3,CWIN,HospitalUnit,14 bed intensive care ward,NaN,0,0.0,NaN,NaN,NaN,NaN
2,4,INBD,HospitalUnit,9 bed intensive care ward,NaN,0,0.0,NaN,NaN,NaN,NaN
3,5,INID,HospitalUnit,"9 bed mixed ward (intermediate care, non-invas...",NaN,0,0.0,NaN,NaN,NaN,NaN
4,15,MVImporter,Importer,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1617,3141,Oxyflow Flow,SignalFloat,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1618,3142,Optiflow O2,SignalFloat,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1619,3143,Optiflow Flow,SignalFloat,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1620,3144,O2 Flow,SignalFloat,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [8]:
data_float

,id,CaseID,DataID,Offset,Val,cnt,rawdata
0,1,508837,724,0,83.4750,40,0x00000000000000000000B0420000A2420000A4420000...
1,2,508837,708,0,84.2500,40,0x00000000000000000000A4420000000000009E420000...
2,3,508837,710,0,96.4600,40,0x00000000000000003333AE4200000000CDCCC242CDCC...
3,4,508837,2274,0,18.2821,39,0x0000000000000000000098410000B8410000E8410000...
4,5,508837,707,0,83.5750,40,0x00000000000000000000AC420000A64200009A420000...
...,...,...,...,...,...,...,...
38033956,38206222,352487,3142,208800,45.0000,1,NaN
38033957,38206223,352487,3143,212400,40.0000,1,NaN
38033958,38206224,352487,3142,212400,40.0000,1,NaN
38033959,38206225,352487,3142,219600,45.0000,1,NaN


In [6]:
patient_list = pd.unique(case_data["PatientID"])
icu_ad_list = pd.unique(case_data["CaseID"])


[len(patient_list), len(icu_ad_list)]

[21566, 27350]

There are 21,566 patients with 27,350 ICU admissions.

In [7]:
Counter(d_reference['ReferenceName'])

Counter({'Drug': 622,
         'Laboratory': 539,
         'Fluid': 147,
         'SignalFloat': 73,
         'SurgicalSite': 40,
         'ReferringUnit': 28,
         'DischargeUnit': 28,
         'ProcessedFields': 20,
         'DrugDosagePerHour': 15,
         'Food': 10,
         'HospitalUnit': 9,
         'DischargeType': 8,
         'RespiratorSetting': 7,
         'ExcludedReason': 5,
         'Importer': 4,
         'DrugGivenState': 4,
         'AKI_KDIGO': 4,
         'NYHA': 4,
         'KDIGO_AKI': 4,
         'SurgicalAdmissionType': 4,
         'Sex': 3,
         'YesNoUnknown': 3,
         'FoodKcal': 3,
         'DischargeState': 3,
         'Drainage': 3,
         'Line': 3,
         'GastricTube': 3,
         'Scores': 3,
         'AdmissionUrgency': 3,
         'Airway': 2,
         'SignalInt': 2,
         'SignalDrug': 2,
         'custom_1': 2,
         'IntakePerHour': 2,
         'UrinaryDiversion': 2,
         'EstimatedSurvivalObservationTime': 2,
         '

In [11]:
d_reference.columns

Index(['ReferenceGlobalID', 'ReferenceValue', 'ReferenceName',
       'ReferenceDescription', 'ReferenceUnit', 'ReferenceOrder',
       'ReferenceType', 'Data', 'LOINC_code', 'LOINC_short', 'LOINC_long'],
      dtype='object')

In [38]:
list(d_reference[d_reference['ReferenceName'] == 'Laboratory']['ReferenceValue'])

['Harnelphor Alpha1-Globuline (ZL)',
 'Harnelphor Alpha2-Globuline (ZL)',
 'Alpha-1-Globuline (ZL)',
 'Alpha-2-Globuline (ZL)',
 'ADAMTS-13 (ZL)',
 'Albumin (ZL)',
 'Harnelektrophorese/Albumin (ZL)',
 'Antithrombin III (ZL)',
 'Basophile (ZL)',
 'Basophile, Handdiff. (ZL)',
 'Beta-1-Globuline (ZL)',
 'Beta-2-Globuline (ZL)',
 'Harnelphor Beta-Globuline (ZL)',
 'Frakt.CO-Hämoglobin (ZL)',
 'Frakt.Deoxy-Hämoglobin (ZL)',
 'Frakt.Met-Hämoglobin (ZL)',
 'Frakt.Oxyhämoglobin (ZL)',
 'Hämatokrit (BG) (ZL)',
 'sO2 (ZL)',
 'Blasten (ZL)',
 'C-1-Inhibitor (Aktivität) (ZL)',
 'Kollagen Bindungsaktivität (ZL)',
 'CD16+CD56+/NK-Zellen (ZL)',
 'CD19+/B-Lymphocyten (ZL)',
 'CD3+/T-Lymphocyten (ZL)',
 'CD3+CD4+/T-Helperzellen (ZL)',
 'CD3+CD8+/T-Suppressorzellen (ZL)',
 'HLADR pos.T-Lymphocyten (ZL)',
 'Karbohydrat-def.Transferrin (ZL)',
 'CK-MB Anteil (ZL)',
 'CO-Hämoglobin (ZL)',
 'Eosinophile (ZL)',
 'Eosinophile, Handdiff. (ZL)',
 'Dysmorphe Ery im Sediment (ZL)',
 'Fraktionelle HST-Ausscheidung 

In [53]:
# Filter the DataFrame to find the specific itemid
filter_df = d_reference[d_reference['ReferenceValue'].str.contains('Natrium', case = False, na = False)]

# Print the itemid(s) for specific attribute
print(filter_df[['ReferenceGlobalID', 'ReferenceValue', 'ReferenceName']])

      ReferenceGlobalID                                     ReferenceValue  \
286                 447                        Natrium im 24-Std.Harn (ZL)   
294                 455                                  Natrium (BG) (ZL)   
308                 469                                       Natrium (ZL)   
309                 470                        Natrium im Spontanharn (ZL)   
525                 686                                      Natrium (BGA)   
693                1475                                    Natriumvalproat   
699                1481                                  Natriumpicosulfat   
702                1484               ZzNatrium- und Megluminanmidotrizoat   
710                1492                                 Natrium perchlorat   
772                1554                                Natriumnitroprussid   
773                1555                              ZzNatriumchlorid 0,9%   
782                1564                          Natriumpolystyr

In [48]:
d_reference[d_reference['ReferenceGlobalID'] == 367]['ReferenceValue']

206    Kreatinin (ZL)
Name: ReferenceValue, dtype: object

In [ ]:
id_list = [707, 708, 724, 719, 2274, 2280, 2282, 701, 702, 703, 704, 705, 706, 710, 727, 289, 658, 206, 237, 468, 455, 469, 686]

In [ ]:
variable_list = VitalSigns_id + GCS_score_id + Vent_para_id + Labs_id + General_id + ADT_id + add_id_1

d_items_data_1 = d_items_data[d_items_data['itemid'].isin(variable_list)].copy()
len(d_items_data_1)

In [ ]:
item_id_list = d_items_data_1['itemid'].tolist()

In [ ]:
# Define column data types explicitly
dtypes = {
    'cgid': 'float64',
    'stay_id': 'float64',
    'error': 'float64',
    'resultstatus': 'object',
    'stopped': 'object',
    'value': 'object',
    'valuenum': 'float64',
    'warning': 'float64',
    'valueuom': 'object',
    'caregiver_id': 'float64'
}

# Read the CSV file using Dask
chart_events = dd.read_csv(
    '../icu_data/mimic_iv/chartevents.csv.gz',
    dtype = dtypes,
    compression = 'gzip',
    assume_missing = True,
    blocksize = None
)

chart_events = chart_events[chart_events.itemid.isin(item_id_list)]

# Compute the Dask DataFrame into a Pandas DataFrame with progress monitoring
try:
    with ProgressBar():
        chart_events_data = chart_events.compute()
    print("Data successfully loaded!")
except Exception as e:
    print(f"Error computing DataFrame: {e}")

In [ ]:
chart_events_data

In [ ]:
chart_events_data.to_csv('../icu_data/mimic_iv/chart_events_data.csv', index = False)

In [ ]:
ICU_unit = ['Medical Intensive Care Unit (MICU)', 
            'Surgical Intensive Care Unit (SICU)', 
            'Medical/Surgical Intensive Care Unit (MICU/SICU)', 
            'Cardiac Vascular Intensive Care Unit (CVICU)', 
            'Coronary Care Unit (CCU)', 
            'Trauma SICU (TSICU)']

ICU_patient_data_test = ICU_patient_data[ICU_patient_data['first_careunit'].isin(ICU_unit)].copy()

In [ ]:
chart_events_data_1 = chart_events_data[chart_events_data['stay_id'].isin(ICU_patient_data_test['stay_id'])].copy()

In [ ]:
chart_events_data_1

In [ ]:
d_items_data_1['linksto'].value_counts()

In [ ]:
d_items_data_chart = d_items_data_1[d_items_data_1['linksto'] == 'chartevents'].copy()
d_items_data_output = d_items_data_1[d_items_data_1['linksto'] == 'outputevents'].copy()
d_items_data_datetime = d_items_data_1[d_items_data_1['linksto'] == 'datetimeevents'].copy()
d_items_data_ingred = d_items_data_1[d_items_data_1['linksto'] == 'ingredientevents'].copy()

In [ ]:
d_items_data_chart

- Select and mark patients

In [ ]:
ICU_unit = ['Medical Intensive Care Unit (MICU)', 
            'Surgical Intensive Care Unit (SICU)', 
            'Medical/Surgical Intensive Care Unit (MICU/SICU)', 
            'Cardiac Vascular Intensive Care Unit (CVICU)', 
            'Coronary Care Unit (CCU)', 
            'Trauma SICU (TSICU)']

ICU_patient_data_test = ICU_patient_data[ICU_patient_data['first_careunit'].isin(ICU_unit)].copy()

In [ ]:
ICU_patient_data_s_15 = ICU_patient_data_test.copy()
# ICU_patient_data_s_15 = ICU_patient_data_test[ICU_patient_data_test['los'] <= 15.00].copy()
# ICU_patient_data_s_30 = ICU_patient_data_test[ICU_patient_data_test['los'] <= 30.00].copy()

In [ ]:
ICU_patient_data_s_15 = ICU_patient_data_s_15.reset_index(drop = True)

In [ ]:
chart_events_data_1 = chart_events_data_1.reset_index(drop = True)

In [ ]:
ICU_patient_data_s_15['intime'] = pd.to_datetime(ICU_patient_data_s_15['intime'])
ICU_patient_data_s_15['outtime'] = pd.to_datetime(ICU_patient_data_s_15['outtime'])

ICU_patient_data_s_15['TD_LOS'] = ICU_patient_data_s_15['outtime'] - ICU_patient_data_s_15['intime']

In [ ]:
ICU_patient_data_s_15 = ICU_patient_data_s_15.sort_values(by = ['subject_id', 'intime'])

pa_list = pd.unique(ICU_patient_data_s_15['subject_id'])
icu_list = pd.unique(ICU_patient_data_s_15['stay_id'])

# build the readmission list
icu_rd_list = []

for i in range(len(pa_list)):
    sub_data = ICU_patient_data_s_15[ICU_patient_data_s_15['subject_id'] == pa_list[i]]
    if len(pd.unique(sub_data['stay_id'])) > 1:
        icu_rd_list.append(pa_list[i])

ICU_patient_data_rd = ICU_patient_data_s_15[ICU_patient_data_s_15['subject_id'].isin(icu_rd_list)].copy()

pa_list_d_7 = []
pa_list_d_14 = []
pa_list_d_21 = []
pa_list_d_30 = []
pa_list_d_60 = []
pa_list_d_90 = []

icu_rd_7_list = []
icu_rd_14_list = []
icu_rd_21_list = []
icu_rd_30_list = []
icu_rd_60_list = []
icu_rd_90_list = []

dist_fail_7_list = []
dist_fail_14_list = []
dist_fail_21_list = []
dist_fail_30_list = []
dist_fail_60_list = []
dist_fail_90_list = []

for i in tqdm(range(len(icu_rd_list))):
    sub_data = ICU_patient_data_rd[ICU_patient_data_rd['subject_id'] == icu_rd_list[i]]
    
    for j in range(1, len(sub_data)):
        if sub_data['stay_id'].iloc[j] != sub_data['stay_id'].iloc[j-1]:

            if sub_data['intime'].iloc[j] - sub_data['outtime'].iloc[j-1] <= pd.Timedelta('7 days 00:00:00'):
                pa_list_d_7.append(icu_rd_list[i])
                dist_fail_7_list.append(sub_data['stay_id'].iloc[j - 1])
                icu_rd_7_list.append(sub_data['stay_id'].iloc[j])
                
            if sub_data['intime'].iloc[j] - sub_data['outtime'].iloc[j-1] <= pd.Timedelta('14 days 00:00:00'):
                pa_list_d_14.append(icu_rd_list[i])
                dist_fail_14_list.append(sub_data['stay_id'].iloc[j - 1])
                icu_rd_14_list.append(sub_data['stay_id'].iloc[j])

            if sub_data['intime'].iloc[j] - sub_data['outtime'].iloc[j-1] <= pd.Timedelta('21 days 00:00:00'):
                pa_list_d_21.append(icu_rd_list[i])
                dist_fail_21_list.append(sub_data['stay_id'].iloc[j - 1])
                icu_rd_21_list.append(sub_data['stay_id'].iloc[j])

            if sub_data['intime'].iloc[j] - sub_data['outtime'].iloc[j-1] <= pd.Timedelta('30 days 00:00:00'):
                pa_list_d_30.append(icu_rd_list[i])
                dist_fail_30_list.append(sub_data['stay_id'].iloc[j - 1])
                icu_rd_30_list.append(sub_data['stay_id'].iloc[j])

            if sub_data['intime'].iloc[j] - sub_data['outtime'].iloc[j-1] <= pd.Timedelta('60 days 00:00:00'):
                pa_list_d_60.append(icu_rd_list[i])
                dist_fail_60_list.append(sub_data['stay_id'].iloc[j - 1])
                icu_rd_60_list.append(sub_data['stay_id'].iloc[j])

            if sub_data['intime'].iloc[j] - sub_data['outtime'].iloc[j-1] <= pd.Timedelta('90 days 00:00:00'):
                pa_list_d_90.append(icu_rd_list[i])
                dist_fail_90_list.append(sub_data['stay_id'].iloc[j - 1])
                icu_rd_90_list.append(sub_data['stay_id'].iloc[j])      
        
        else:
            print("Error: ", sub_data['stay_id'].iloc[j])          

In [ ]:
ICU_patient_data_s_15['discharge_fail_7_day'] = 0
ICU_patient_data_s_15['discharge_fail_14_day'] = 0
ICU_patient_data_s_15['discharge_fail_21_day'] = 0
ICU_patient_data_s_15['discharge_fail_30_day'] = 0
ICU_patient_data_s_15['discharge_fail_60_day'] = 0
ICU_patient_data_s_15['discharge_fail_90_day'] = 0

ICU_patient_data_s_15['readmission_7_day'] = 0
ICU_patient_data_s_15['readmission_14_day'] = 0
ICU_patient_data_s_15['readmission_21_day'] = 0
ICU_patient_data_s_15['readmission_30_day'] = 0
ICU_patient_data_s_15['readmission_60_day'] = 0
ICU_patient_data_s_15['readmission_90_day'] = 0


# Mark discharge failures for each time window
for stay_id in dist_fail_7_list:
    ICU_patient_data_s_15.loc[ICU_patient_data_s_15['stay_id'] == stay_id, 'discharge_fail_7_day'] = 1

for stay_id in dist_fail_14_list:
    ICU_patient_data_s_15.loc[ICU_patient_data_s_15['stay_id'] == stay_id, 'discharge_fail_14_day'] = 1

for stay_id in dist_fail_21_list:
    ICU_patient_data_s_15.loc[ICU_patient_data_s_15['stay_id'] == stay_id, 'discharge_fail_21_day'] = 1

for stay_id in dist_fail_30_list:
    ICU_patient_data_s_15.loc[ICU_patient_data_s_15['stay_id'] == stay_id, 'discharge_fail_30_day'] = 1

for stay_id in dist_fail_60_list:
    ICU_patient_data_s_15.loc[ICU_patient_data_s_15['stay_id'] == stay_id, 'discharge_fail_60_day'] = 1

for stay_id in dist_fail_90_list:
    ICU_patient_data_s_15.loc[ICU_patient_data_s_15['stay_id'] == stay_id, 'discharge_fail_90_day'] = 1


# Mark readmissions for each time window
for stay_id in icu_rd_7_list:
    ICU_patient_data_s_15.loc[ICU_patient_data_s_15['stay_id'] == stay_id, 'readmission_7_day'] = 1

for stay_id in icu_rd_14_list:
    ICU_patient_data_s_15.loc[ICU_patient_data_s_15['stay_id'] == stay_id, 'readmission_14_day'] = 1
    
for stay_id in icu_rd_21_list:
    ICU_patient_data_s_15.loc[ICU_patient_data_s_15['stay_id'] == stay_id, 'readmission_21_day'] = 1

for stay_id in icu_rd_30_list:
    ICU_patient_data_s_15.loc[ICU_patient_data_s_15['stay_id'] == stay_id, 'readmission_30_day'] = 1

for stay_id in icu_rd_60_list:
    ICU_patient_data_s_15.loc[ICU_patient_data_s_15['stay_id'] == stay_id, 'readmission_60_day'] = 1

for stay_id in icu_rd_90_list:
    ICU_patient_data_s_15.loc[ICU_patient_data_s_15['stay_id'] == stay_id, 'readmission_90_day'] = 1

In [ ]:
ICU_patient_data_s_15[ICU_patient_data_s_15['subject_id'] == 16133115]

In [ ]:
admission_data = pd.read_csv('../icu_data/mimic_iv/admissions.csv.gz', compression = 'gzip')
patients_data = pd.read_csv('../icu_data/mimic_iv/patients.csv.gz', compression = 'gzip')

In [ ]:
patients_data_select = patients_data.drop(columns = ['anchor_year', 'anchor_year_group'])
admission_data_select = admission_data[['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'race']]

patients_data_select = patients_data_select[patients_data_select['subject_id'].isin(ICU_patient_data_s_15['subject_id'])]
admission_data_select = admission_data_select[admission_data_select['subject_id'].isin(ICU_patient_data_s_15['subject_id'])]

admission_data_select_v1 = admission_data_select[['subject_id', 'race']].copy()
admission_data_select_v1 = admission_data_select_v1.drop_duplicates(subset = ['subject_id'], keep = 'first')

ICU_patient_data_s15_v1 = pd.merge(ICU_patient_data_s_15, admission_data_select_v1, on = 'subject_id', how = 'left')
ICU_patient_data_s15_v2 = pd.merge(ICU_patient_data_s15_v1, patients_data_select, on = 'subject_id', how = 'left')

In [ ]:
ICU_patient_data_s15_v2.info()

In [ ]:
ICU_patient_data_s15_v2['dod'] = pd.to_datetime(ICU_patient_data_s15_v2['dod'])
ICU_patient_data_s15_v2['TD_death_disch'] = ICU_patient_data_s15_v2['dod'] - ICU_patient_data_s15_v2['outtime']

ICU_patient_data_s15_v2['death_in_ICU'] = 0

ICU_patient_data_s15_v2['death_out_ICU_7_day'] = 0
ICU_patient_data_s15_v2['death_out_ICU_14_day'] = 0
ICU_patient_data_s15_v2['death_out_ICU_21_day'] = 0
ICU_patient_data_s15_v2['death_out_ICU_30_day'] = 0
ICU_patient_data_s15_v2['death_out_ICU_60_day'] = 0
ICU_patient_data_s15_v2['death_out_ICU_90_day'] = 0


# Mark death in ICU
ICU_patient_data_s15_v2.loc[ICU_patient_data_s15_v2['TD_death_disch'] <= pd.Timedelta(0), 'death_in_ICU'] = 1

# Mark death after ICU discharge within different time windows
ICU_patient_data_s15_v2.loc[(ICU_patient_data_s15_v2['TD_death_disch'] > pd.Timedelta(0)) & 
                           (ICU_patient_data_s15_v2['TD_death_disch'] <= pd.Timedelta(days = 7)), 'death_out_ICU_7_day'] = 1

ICU_patient_data_s15_v2.loc[(ICU_patient_data_s15_v2['TD_death_disch'] > pd.Timedelta(0)) &
                           (ICU_patient_data_s15_v2['TD_death_disch'] <= pd.Timedelta(days = 14)), 'death_out_ICU_14_day'] = 1

ICU_patient_data_s15_v2.loc[(ICU_patient_data_s15_v2['TD_death_disch'] > pd.Timedelta(0)) &
                           (ICU_patient_data_s15_v2['TD_death_disch'] <= pd.Timedelta(days = 21)), 'death_out_ICU_21_day'] = 1

ICU_patient_data_s15_v2.loc[(ICU_patient_data_s15_v2['TD_death_disch'] > pd.Timedelta(0)) &
                           (ICU_patient_data_s15_v2['TD_death_disch'] <= pd.Timedelta(days = 30)), 'death_out_ICU_30_day'] = 1

ICU_patient_data_s15_v2.loc[(ICU_patient_data_s15_v2['TD_death_disch'] > pd.Timedelta(0)) &
                           (ICU_patient_data_s15_v2['TD_death_disch'] <= pd.Timedelta(days = 60)), 'death_out_ICU_60_day'] = 1

ICU_patient_data_s15_v2.loc[(ICU_patient_data_s15_v2['TD_death_disch'] > pd.Timedelta(0)) &
                           (ICU_patient_data_s15_v2['TD_death_disch'] <= pd.Timedelta(days = 90)), 'death_out_ICU_90_day'] = 1

- Mark readmission count

In [ ]:
list(ICU_patient_data_s15_v2.columns)

In [ ]:
patient_list = pd.unique(ICU_patient_data_s15_v2['subject_id'])
icu_stay_list = pd.unique(ICU_patient_data_s15_v2['stay_id'])

In [ ]:
time_windows = [7, 14, 21, 30, 60, 90]

for w in time_windows:
    ICU_patient_data_s15_v2[f'readmission_count_{w}_day'] = 0

patient_list = ICU_patient_data_s15_v2['subject_id'].unique()

for patient_id in patient_list:
    sub_data = ICU_patient_data_s15_v2.loc[ICU_patient_data_s15_v2['subject_id'] == patient_id]

    prev_counts = {w: 0 for w in time_windows}

    for idx, row in sub_data.iterrows():

        for w in time_windows:

            if row[f'readmission_{w}_day'] == 1:
                current_count = prev_counts[w] + 1
            
            else:
                # current_count = prev_counts[w]
                current_count = 0

            ICU_patient_data_s15_v2.at[idx, f'readmission_count_{w}_day'] = current_count

            prev_counts[w] = current_count

In [ ]:
ICU_patient_data_s15_v2[ICU_patient_data_s15_v2['subject_id'] == 16133115][['stay_id', 'discharge_fail_30_day', 'readmission_count_30_day']]

In [ ]:
list(ICU_patient_data_s15_v2.columns)

In [ ]:
len(list(ICU_patient_data_s15_v2.columns))

In [ ]:
ICU_patient_data_s15_v2[['subject_id', 'stay_id', 'readmission_30_day', 'readmission_count_30_day']].head(50)

- chart_events Data Preprocessing

In [ ]:
d_items_data_chart_numeric = d_items_data_chart[d_items_data_chart['param_type'] == 'Numeric']
d_items_data_chart_text = d_items_data_chart[d_items_data_chart['param_type'] == 'Text']
d_items_data_chart_numeric_tag = d_items_data_chart[d_items_data_chart['param_type'] == 'Numeric with tag']
d_items_data_chart_checkbox = d_items_data_chart[d_items_data_chart['param_type'] == 'Checkbox']

In [ ]:
d_items_data_chart_select = d_items_data_chart[~d_items_data_chart['label'].isin(['Ventilator Type', 'Ventilator Mode', 
                                                                                  'SaO2 < 90% > 2 min', 'Gender', 
                                                                                  'Race', 'Cardiovascular', 'Musculoskeletal', 
                                                                                  'Neurological', 'Nutrition', 'Respiratory', 
                                                                                  'Vascular', 'Mechanically Ventilated', 
                                                                                  'Re-admit < 48 hours'])]

In [ ]:
chart_events_data_1[['subject_id', 'hadm_id', 'stay_id', 'itemid']] = chart_events_data_1[['subject_id', 'hadm_id', 'stay_id', 'itemid']].astype('int64')

In [ ]:
chart_events_data_1.info()

In [ ]:
chart_events_data_2 = chart_events_data_1[chart_events_data_1['itemid'].isin(d_items_data_chart_select['itemid'])]

In [ ]:
chart_events_data_2 = chart_events_data_2.reset_index(drop = True)

In [ ]:
chart_events_data_2.info()

In [ ]:
tuple_list = list(zip(d_items_data_chart_select["itemid"], d_items_data_chart_select["label"]))
print(tuple_list)

In [ ]:
chart_events_data_2[chart_events_data_2['itemid'] == 228878]

In [ ]:
# Counter(chart_events_data_2[chart_events_data_2['itemid'] == 220048]['value'])

In [ ]:
var_delete_list = ['Heart Rhythm', 'PA %O2 Saturation (PA Line)', 'SOFA Score', 'Urine output_ApacheIV']

In [ ]:
d_items_data_chart_select = d_items_data_chart_select[~d_items_data_chart_select['label'].isin(var_delete_list)]

In [ ]:
d_items_data_chart_select

In [ ]:
# physio_table_7_day = {'subject_id':[], 'hadm_id':[], 'stay_id':[], 
#                       'time':[],
#                       'icu_starttime':[], 'icu_endtime':[], 'los':[],
#                       'discharge_fail':[], 
#                       'readmission':[], 'readmission_count':[],
#                       'death_in_ICU':[], 'death_out_ICU':[], 
#                       'age':[], 'gender':[], 'race':[]}

# for label in d_items_data_chart_select['label']:
#     physio_table_7_day[label] = []

# physio_table_7_day

In [ ]:
[len(pd.unique(chart_events_data_2['stay_id'])), len(pd.unique(ICU_patient_data_s15_v2['stay_id']))]

In [ ]:
chart_events_data_3 = chart_events_data_2[chart_events_data_2['stay_id'].isin(ICU_patient_data_s15_v2['stay_id'])].copy()

In [ ]:
[len(pd.unique(chart_events_data_3['stay_id'])), len(pd.unique(ICU_patient_data_s15_v2['stay_id']))]

In [ ]:
[len(pd.unique(chart_events_data_3['subject_id'])), len(pd.unique(ICU_patient_data_s15_v2['subject_id']))]

In [ ]:
drop_patient_list = pd.unique(ICU_patient_data_s15_v2[~ICU_patient_data_s15_v2['stay_id'].isin(chart_events_data_3['stay_id'])]['subject_id'])

In [ ]:
drop_patient_list

In [ ]:
ICU_patient_data_s15_v2 = ICU_patient_data_s15_v2[~ICU_patient_data_s15_v2['subject_id'].isin(drop_patient_list)]

In [ ]:
ICU_patient_data_s15_v2.info()

In [ ]:
drop_patient_list = pd.unique(ICU_patient_data_s15_v2[ICU_patient_data_s15_v2['los'].isnull()]['subject_id'])

In [ ]:
drop_patient_list

In [ ]:
ICU_patient_data_s15_v2 = ICU_patient_data_s15_v2[~ICU_patient_data_s15_v2['subject_id'].isin(drop_patient_list)]

In [ ]:
chart_events_data_3 = chart_events_data_3[chart_events_data_3['stay_id'].isin(ICU_patient_data_s15_v2['stay_id'])].copy()

In [ ]:
[len(pd.unique(chart_events_data_3['stay_id'])), len(pd.unique(ICU_patient_data_s15_v2['stay_id']))]

In [ ]:
chart_events_data_3 = chart_events_data_3.reset_index(drop = True)

In [ ]:
chart_events_data_3.info()

In [ ]:
ICU_patient_data_s15_v2 = ICU_patient_data_s15_v2.reset_index(drop = True)

In [ ]:
ICU_patient_data_s15_v2.info()

In [ ]:
icu_stay_list = list(ICU_patient_data_s15_v2['stay_id'])

There are ten ICU admissions without any char_event records.

In [ ]:
chart_events_data_3['charttime'] = pd.to_datetime(chart_events_data_3['charttime'])
chart_events_data_3['storetime'] = pd.to_datetime(chart_events_data_3['storetime'])

In [ ]:
chart_events_data_3.info()

In [ ]:
# Counter(ICU_patient_data_s15_v2['readmission_count_7_day'])

In [ ]:
# ICU_patient_data_s15_v2['readmission_count_7_day'].to_numpy().max()

In [ ]:
def data_select(data, i_1, i_2, i_3):
    sub_data = data.loc[(data['charttime'] >= i_1) & 
                        (data['charttime'] <= i_2) & 
                        (data["itemid"] == i_3)]
    return sub_data

In [ ]:
# for i in range(len(icu_stay_list)):
    
#     print("The number of processed ICU stay admissions: ", i)
     
#     index = ICU_patient_data_s15_v2["intime"].iloc[i]
    
#     s_table_id = chart_events_data_3[chart_events_data_3['stay_id'] == icu_stay_list[i]]

#     while index <= ICU_patient_data_s15_v2["outtime"].iloc[i]:
#         physio_table_7_day['subject_id'].append(ICU_patient_data_s15_v2['subject_id'].iloc[i])
#         physio_table_7_day['hadm_id'].append(ICU_patient_data_s15_v2['hadm_id'].iloc[i])
#         physio_table_7_day['stay_id'].append(ICU_patient_data_s15_v2['stay_id'].iloc[i])
#         physio_table_7_day['icu_starttime'].append(ICU_patient_data_s15_v2['intime'].iloc[i])
#         physio_table_7_day['icu_endtime'].append(ICU_patient_data_s15_v2['outtime'].iloc[i]) 
#         physio_table_7_day['los'].append(ICU_patient_data_s15_v2['los'].iloc[i])        
#         physio_table_7_day['discharge_fail'].append(ICU_patient_data_s15_v2['discharge_fail_7_day'].iloc[i])
#         physio_table_7_day['readmission'].append(ICU_patient_data_s15_v2['readmission_7_day'].iloc[i])
#         physio_table_7_day['readmission_count'].append(ICU_patient_data_s15_v2['readmission_count_7_day'].iloc[i])
#         physio_table_7_day['death_in_ICU'].append(ICU_patient_data_s15_v2['death_in_ICU'].iloc[i])
#         physio_table_7_day['death_out_ICU'].append(ICU_patient_data_s15_v2['death_out_ICU_7_day'].iloc[i])
#         physio_table_7_day['age'].append(ICU_patient_data_s15_v2['anchor_age'].iloc[i])
#         physio_table_7_day['gender'].append(ICU_patient_data_s15_v2['gender'].iloc[i])
#         physio_table_7_day['race'].append(ICU_patient_data_s15_v2['race'].iloc[i])
        
#         td = pd.Timedelta('0 days 12:00:00')
#         rd_idx = physio_table_7_day['readmission_count'][-1]
        
#         index_1 = index + td * (0.5**rd_idx)
        
#         if index_1 <= ICU_patient_data_s15_v2["outtime"].iloc[i]:
#             physio_table_7_day['time'].append(index_1)
#         else:
#             index_1 = ICU_patient_data_s15_v2["outtime"].iloc[i]
#             physio_table_7_day['time'].append(index_1)
            
#         for j in range(len(d_items_data_chart_select)):
#             s_table = data_select(s_table_id, 
#                                   index, 
#                                   index_1,
#                                   d_items_data_chart_select["itemid"].iloc[j])

#             n = len(s_table)

#             if n >= 1:
#                 physio_table_7_day[d_items_data_chart_select['label'].iloc[j]].append(s_table['valuenum'].mean())
#                 # physio_table[d_items_data_chart_select['label'].iloc[j]].append(s_table['valuenum'].iloc[-1])

#             else:
#                 physio_table_7_day[d_items_data_chart_select['label'].iloc[j]].append(np.nan)

#         index = index + td * (0.5**rd_idx)

In [ ]:
# physio_df_7d = pd.DataFrame.from_dict(physio_table_7_day)
# physio_df_7d.to_csv('../icu_data/mimic_iv/physio_df_7d.csv', index = False)

In [ ]:
physio_table_30_day = {'subject_id':[], 'hadm_id':[], 'stay_id':[], 
                      'time':[],
                      'icu_starttime':[], 'icu_endtime':[], 'los':[],
                      'discharge_fail':[], 
                      'readmission':[], 'readmission_count':[],
                      'death_in_ICU':[], 'death_out_ICU':[], 
                      'age':[], 'gender':[], 'race':[]}

for label in d_items_data_chart_select['label']:
    physio_table_30_day[label] = []

physio_table_30_day

In [ ]:
ICU_patient_data_s15_v2['readmission_count_30_day'].to_numpy().max()

In [ ]:
Counter(ICU_patient_data_s15_v2['readmission_count_30_day'])

In [ ]:
ICU_patient_data_s15_v2[ICU_patient_data_s15_v2['readmission_count_30_day'] == 35]

In [ ]:
ICU_patient_data_s15_v2[ICU_patient_data_s15_v2['subject_id'] == 18358138]

In [ ]:
for i in range(len(icu_stay_list)):
    
    print("The number of processed ICU stay admissions: ", i)
     
    index = ICU_patient_data_s15_v2["intime"].iloc[i]
    
    s_table_id = chart_events_data_3[chart_events_data_3['stay_id'] == icu_stay_list[i]]

    while index <= ICU_patient_data_s15_v2["outtime"].iloc[i]:
        physio_table_30_day['subject_id'].append(ICU_patient_data_s15_v2['subject_id'].iloc[i])
        physio_table_30_day['hadm_id'].append(ICU_patient_data_s15_v2['hadm_id'].iloc[i])
        physio_table_30_day['stay_id'].append(ICU_patient_data_s15_v2['stay_id'].iloc[i])
        physio_table_30_day['icu_starttime'].append(ICU_patient_data_s15_v2['intime'].iloc[i])
        physio_table_30_day['icu_endtime'].append(ICU_patient_data_s15_v2['outtime'].iloc[i]) 
        physio_table_30_day['los'].append(ICU_patient_data_s15_v2['los'].iloc[i])        
        physio_table_30_day['discharge_fail'].append(ICU_patient_data_s15_v2['discharge_fail_30_day'].iloc[i])
        physio_table_30_day['readmission'].append(ICU_patient_data_s15_v2['readmission_30_day'].iloc[i])
        physio_table_30_day['readmission_count'].append(ICU_patient_data_s15_v2['readmission_count_30_day'].iloc[i])
        physio_table_30_day['death_in_ICU'].append(ICU_patient_data_s15_v2['death_in_ICU'].iloc[i])
        physio_table_30_day['death_out_ICU'].append(ICU_patient_data_s15_v2['death_out_ICU_30_day'].iloc[i])
        physio_table_30_day['age'].append(ICU_patient_data_s15_v2['anchor_age'].iloc[i])
        physio_table_30_day['gender'].append(ICU_patient_data_s15_v2['gender'].iloc[i])
        physio_table_30_day['race'].append(ICU_patient_data_s15_v2['race'].iloc[i])
        
        td = pd.Timedelta('0 days 12:00:00')
        rd_idx = physio_table_30_day['readmission_count'][-1]
        
        if rd_idx <= 4:
        
            index_1 = index + td * (0.5**rd_idx)

            if index_1 <= ICU_patient_data_s15_v2["outtime"].iloc[i]:
                physio_table_30_day['time'].append(index_1)
            else:
                index_1 = ICU_patient_data_s15_v2["outtime"].iloc[i]
                physio_table_30_day['time'].append(index_1)

            for j in range(len(d_items_data_chart_select)):
                s_table = data_select(s_table_id, 
                                      index, 
                                      index_1,
                                      d_items_data_chart_select["itemid"].iloc[j])

                n = len(s_table)

                if n >= 1:
                    physio_table_30_day[d_items_data_chart_select['label'].iloc[j]].append(s_table['valuenum'].mean())
                    # physio_table[d_items_data_chart_select['label'].iloc[j]].append(s_table['valuenum'].iloc[-1])

                else:
                    physio_table_30_day[d_items_data_chart_select['label'].iloc[j]].append(np.nan)

            index = index + td * (0.5**rd_idx)
            
        else:
            rd_idx = 4
            index_1 = index + td * (0.5**rd_idx)

            if index_1 <= ICU_patient_data_s15_v2["outtime"].iloc[i]:
                physio_table_30_day['time'].append(index_1)
            else:
                index_1 = ICU_patient_data_s15_v2["outtime"].iloc[i]
                physio_table_30_day['time'].append(index_1)

            for j in range(len(d_items_data_chart_select)):
                s_table = data_select(s_table_id, 
                                      index, 
                                      index_1,
                                      d_items_data_chart_select["itemid"].iloc[j])

                n = len(s_table)

                if n >= 1:
                    physio_table_30_day[d_items_data_chart_select['label'].iloc[j]].append(s_table['valuenum'].mean())
                    # physio_table[d_items_data_chart_select['label'].iloc[j]].append(s_table['valuenum'].iloc[-1])

                else:
                    physio_table_30_day[d_items_data_chart_select['label'].iloc[j]].append(np.nan)

            index = index + td * (0.5**rd_idx)

In [ ]:
physio_df_30d = pd.DataFrame.from_dict(physio_table_30_day)

In [ ]:
physio_df_30d.info()

In [ ]:
physio_df_30d[physio_df_30d['subject_id'] == 16133115]

In [ ]:
physio_df_30d[physio_df_30d['subject_id'] == 16133115][['subject_id', 'stay_id', 'discharge_fail', 'readmission', 'readmission_count']].iloc[60:100]


In [ ]:
# physio_df_30d.to_csv('../icu_data/mimic_iv/physio_df_30d.csv', index = False)

## Data Preprocess - Part 2

In [ ]:
physio_df = physio_df_30d.copy()

In [ ]:
physio_df

- Tidal Volume

In [ ]:
physio_df['Tidal Volume (set)'] = physio_df['Tidal Volume (set)']/1000
physio_df['Tidal Volume (observed)'] = physio_df['Tidal Volume (observed)']/1000
physio_df['Tidal Volume (spontaneous)'] = physio_df['Tidal Volume (spontaneous)']/1000

- Time information

In [ ]:
physio_df['Tidal Volume (set)'] = physio_df['Tidal Volume (set)']/1000
physio_df['Tidal Volume (observed)'] = physio_df['Tidal Volume (observed)']/1000
physio_df['Tidal Volume (spontaneous)'] = physio_df['Tidal Volume (spontaneous)']/1000

- Gender information

In [ ]:
gender_dummies = pd.get_dummies(physio_df.gender)
physio_df = pd.concat([physio_df, gender_dummies], axis = 'columns')

In [ ]:
physio_df = physio_df.drop(columns = ['gender', 'F'])

- Race information

In [ ]:
# race_dummies = pd.get_dummies(physio_df_7d.race, prefix='race')
# physio_df_7d = pd.concat([physio_df_7d, race_dummies], axis='columns')


physio_df = physio_df.drop(columns = ['race'])

- Discharge action

In [ ]:
icu_stayid_list = physio_df['stay_id'].unique()

physio_df['discharge_action'] = 0

for i in range(len(icu_stayid_list)):

    time_idx = physio_df[(physio_df['stay_id'] == icu_stayid_list[i])]['time'].iloc[-1]
    
    physio_df.loc[(physio_df['stay_id'] == icu_stayid_list[i]) & (physio_df['time'] == time_idx), 'discharge_action'] = 1

- Blood pressure

In [ ]:
def assign_blood_pressure(row):
    if pd.isna(row['Arterial Blood Pressure systolic']) and not pd.isna(row['Non Invasive Blood Pressure systolic']):
        return row['Non Invasive Blood Pressure systolic']
    elif not pd.isna(row['Arterial Blood Pressure systolic']):
        return row['Arterial Blood Pressure systolic']
    elif not pd.isna(row['ART BP Systolic']):
        return row['ART BP Systolic']
    else:
        return np.nan

physio_df['Blood Pressure Systolic'] = physio_df.apply(assign_blood_pressure, axis = 1)

In [ ]:
def assign_blood_pressure_diastolic(row):
    if pd.isna(row['Arterial Blood Pressure diastolic']) and not pd.isna(row['Non Invasive Blood Pressure diastolic']):
        return row['Non Invasive Blood Pressure diastolic']
    elif not pd.isna(row['Arterial Blood Pressure diastolic']):
        return row['Arterial Blood Pressure diastolic']
    elif not pd.isna(row['ART BP Diastolic']):
        return row['ART BP Diastolic']
    else:
        return np.nan

physio_df['Blood Pressure Diastolic'] = physio_df.apply(assign_blood_pressure_diastolic, axis = 1)

In [ ]:
def assign_blood_pressure_mean(row):
    if pd.isna(row['Arterial Blood Pressure mean']) and not pd.isna(row['Non Invasive Blood Pressure mean']):
        return row['Non Invasive Blood Pressure mean']
    elif not pd.isna(row['Arterial Blood Pressure mean']):
        return row['Arterial Blood Pressure mean']
    elif not pd.isna(row['ART BP Mean']):
        return row['ART BP Mean']
    else:
        return np.nan

physio_df['Blood Pressure Mean'] = physio_df.apply(assign_blood_pressure_mean, axis = 1)

- Temperature

In [ ]:
def assign_temperature(row):
    if pd.isna(row['Temperature Celsius']) and not pd.isna(row['Temperature Fahrenheit']):
        return (row['Temperature Fahrenheit']-32) * 5.0/9.0
    elif not pd.isna(row['Temperature Celsius']):
        return row['Temperature Celsius']
    else:
        return np.nan

physio_df['Temperature C'] = physio_df.apply(assign_temperature, axis = 1)

- O2 Saturation

In [ ]:
def assign_SaO2(row):
    if pd.isna(row['Arterial O2 Saturation']) and not pd.isna(row['O2 saturation pulseoxymetry']):
        return row['O2 saturation pulseoxymetry']
    elif not pd.isna(row['Arterial O2 Saturation']):
        return row['Arterial O2 Saturation']
    else:
        return np.nan

physio_df['SaO2'] = physio_df.apply(assign_SaO2, axis = 1)

- GCS score

In [ ]:
def assign_gcs_score(row):
    return row['GCS - Eye Opening'] + row['GCS - Verbal Response'] + row['GCS - Motor Response']

physio_df['GCS score'] = physio_df.apply(assign_gcs_score, axis = 1)

- PEEP level

In [ ]:
def assign_peep_level(row):
    if pd.isna(row['PEEP set']) and not pd.isna(row['Total PEEP Level']):
        return row['Total PEEP Level']
    elif not pd.isna(row['PEEP set']):
        return row['PEEP set']
    else:
        return np.nan

physio_df['PEEP Level'] = physio_df.apply(assign_peep_level, axis = 1)

- Weight

In [ ]:
def assign_weight(row):
    if not pd.isna(row['Daily Weight']):
        return row['Daily Weight']
    elif not pd.isna(row['Admission Weight (Kg)']):
        return row['Admission Weight (Kg)']
    elif not pd.isna(row['Admission Weight (lbs.)']):
        return row['Admission Weight (lbs.)'] * 0.453592  # Convert lbs to kg
    else:
        return np.nan

physio_df['Weight'] = physio_df.apply(assign_weight, axis = 1)

In [ ]:
physio_df_v1 = physio_df.drop(columns = ['Arterial Blood Pressure systolic', 'Non Invasive Blood Pressure systolic', 'ART BP Systolic', 'Arterial Blood Pressure diastolic', 
                                         'Non Invasive Blood Pressure diastolic', 'ART BP Diastolic', 'Arterial Blood Pressure mean', 'Non Invasive Blood Pressure mean', 
                                         'ART BP Mean', 'Temperature Celsius', 'Temperature Fahrenheit', 'Arterial O2 Saturation', 'O2 saturation pulseoxymetry', 
                                         'GCS - Eye Opening', 'GCS - Verbal Response', 'GCS - Motor Response', 'PEEP set', 'Total PEEP Level', 'Admission Weight (lbs.)', 'Admission Weight (Kg)', 'Daily Weight'])

- Filter out abnormal values

In [ ]:
physio_df_v2 = physio_df_v1.drop(columns = ['Direct Bilirubin', 'PeCO2', 'Creatinine (whole blood)'])

In [ ]:
names_var = ['age',
             'Heart Rate', 'Respiratory Rate', 'Arterial O2 pressure', 'Hemoglobin',
             'Arterial CO2 Pressure', 'PH (Venous)', 'Hematocrit (serum)', 'WBC',
             'Chloride (serum)', 'Creatinine (serum)', 'Glucose (serum)',
             'Magnesium', 'Sodium (serum)', 'PH (Arterial)', 'Inspired O2 Fraction',
             'Tidal Volume (set)', 'Tidal Volume (observed)',
             'Tidal Volume (spontaneous)', 'Respiratory Rate (Set)',
             'Respiratory Rate (spontaneous)', 'Respiratory Rate (Total)',
             'Arterial Base Excess', 'BUN', 'Ionized Calcium', 'Total Bilirubin',
             'Venous CO2 Pressure', 'Venous O2 Pressure', 'Sodium (whole blood)',
             'Chloride (whole blood)', 'Glucose (whole blood)',
             'Hematocrit (whole blood - calc)', 'Potassium (serum)', 'HCO3 (serum)',
             'Albumin', 'Platelet Count', 'Potassium (whole blood)',
             'Prothrombin time', 'PTT', 'INR', 'M',
             'Blood Pressure Systolic', 'Blood Pressure Diastolic',
             'Blood Pressure Mean', 'Temperature C', 'SaO2', 'GCS score', 'PEEP Level', 'Weight']

len(names_var)

In [ ]:
abv_data = physio_df_v2[names_var]

In [ ]:
inspect_col = ['Heart Rate', 'Respiratory Rate', 'Arterial O2 pressure', 'Hemoglobin',
               'Arterial CO2 Pressure', 'PH (Venous)', 'Hematocrit (serum)', 'WBC',
               'Chloride (serum)', 'Creatinine (serum)', 'Glucose (serum)',
               'Magnesium', 'Sodium (serum)', 'PH (Arterial)', 
               'Tidal Volume (observed)',
               'Tidal Volume (spontaneous)', 'Respiratory Rate (Set)',
               'Respiratory Rate (spontaneous)', 'Respiratory Rate (Total)',
               'Arterial Base Excess', 'BUN', 'Ionized Calcium', 'Total Bilirubin',
               'Venous CO2 Pressure', 'Venous O2 Pressure', 'Sodium (whole blood)',
               'Chloride (whole blood)', 'Glucose (whole blood)',
               'Hematocrit (whole blood - calc)', 'Potassium (serum)', 'HCO3 (serum)',
               'Albumin', 'Platelet Count', 'Potassium (whole blood)',
               'Prothrombin time', 'PTT', 'INR', 
               'Blood Pressure Systolic', 'Blood Pressure Diastolic',
               'Blood Pressure Mean', 'Temperature C', 'SaO2', 'Weight']

In [ ]:
ab_data_sub = abv_data[inspect_col]

In [ ]:
# lim = np.logical_or(ab_data_sub >= ab_data_sub.quantile(0.999),
#                     ab_data_sub <= ab_data_sub.quantile(0.001))

q1 = ab_data_sub.quantile(0.25)
q3 = ab_data_sub.quantile(0.75)

iqr = q3 - q1

lower_bound = q1 - 3.0 * iqr
upper_bound = q3 + 3.0 * iqr

In [ ]:
for column in ab_data_sub.columns:
    physio_df_v2.loc[physio_df_v2[column] > upper_bound[column], column] = np.nan
    physio_df_v2.loc[physio_df_v2[column] < lower_bound[column], column] = np.nan

In [ ]:
physio_df_v2.info()

In [ ]:
abv_data = physio_df_v2[names_var]
abv_data.describe().T

## Data Imputation

In [ ]:
drop_list = []

for i in names_var:
    if (physio_df_v2[i].isnull().sum()/len(physio_df_v2)) > 0.75:
        drop_list.append(i)

In [ ]:
middle_list = []

for i in names_var:
    if ((physio_df_v2[i].isnull().sum()/len(physio_df_v2)) <= 0.75) & ((physio_df_v2[i].isnull().sum()/len(physio_df_v2)) >= 0.10):
        middle_list.append(i)

In [ ]:
knn_list = []

for i in names_var:
    if (physio_df_v2[i].isnull().sum()/len(physio_df_v2)) < 0.10:
        knn_list.append(i)

- Forward fill

In [ ]:
for i in range(len(drop_list)):
    physio_df_v2[drop_list[i]] = physio_df_v2.groupby(by = ['stay_id', 'readmission_count'])[drop_list[i]].ffill()

for i in range(len(middle_list)):
    physio_df_v2[middle_list[i]] = physio_df_v2.groupby(by = ['stay_id', 'readmission_count'])[middle_list[i]].ffill()

for i in range(len(knn_list)):
    physio_df_v2[knn_list[i]] = physio_df_v2.groupby(by = ['stay_id', 'readmission_count'])[knn_list[i]].ffill()

In [ ]:
drop_list = []

for i in names_var:
    if (physio_df_v2[i].isnull().sum()/len(physio_df_v2)) > 0.75:
        drop_list.append(i)

In [ ]:
middle_list = []

for i in names_var:
    if ((physio_df_v2[i].isnull().sum()/len(physio_df_v2)) <= 0.75) & ((physio_df_v2[i].isnull().sum()/len(physio_df_v2)) >= 0.10):
        middle_list.append(i)

In [ ]:
knn_list = []

for i in names_var:
    if (physio_df_v2[i].isnull().sum()/len(physio_df_v2)) < 0.10:
        knn_list.append(i)

In [ ]:
physio_df_v3 = physio_df_v2.drop(columns = drop_list)

- Linear interpolation

In [ ]:
feature_list = middle_list + knn_list

In [ ]:
for i in range(len(feature_list)):
    physio_df_v3[feature_list[i]] = physio_df_v3.groupby(by = ['stay_id', 'readmission_count'])[feature_list[i]].apply(lambda x: x.interpolate(method = 'linear'))

In [ ]:
physio_df_v3.columns

In [ ]:
physio_df_v3.info()

In [ ]:
physio_df_v4 = physio_df_v3.drop(columns = ['Venous CO2 Pressure', 'Venous O2 Pressure', 
                                            'Sodium (whole blood)', 'Chloride (whole blood)', 
                                            'Hematocrit (whole blood - calc)', 'Albumin',
                                            'Potassium (whole blood)', 'PEEP Level'])

In [ ]:
physio_df_v4.info()

- KNN Imputation

In [ ]:
import os
from threadpoolctl import threadpool_limits
from joblib import Parallel, delayed
from tqdm import tqdm

In [ ]:
num_threads = os.cpu_count()
print(f"Available CPU threads: {num_threads}")

In [ ]:
imputer = KNNImputer(n_neighbors = 5)

In [ ]:
pro_events_data.columns

In [ ]:
pro_events_data_weight = pro_events_data[['stay_id', 'patientweight']]
pro_events_data_weight = pro_events_data_weight.drop_duplicates(subset = ['stay_id'], keep = 'first')

physio_df_v4 = pd.merge(physio_df_v4, pro_events_data_weight, on = 'stay_id', how = 'left')

In [ ]:
def assign_weight_2(row):
    if not pd.isna(row['Weight']):
        return row['Weight']
    elif not pd.isna(row['patientweight']):
        return row['patientweight']
    else:
        return np.nan

physio_df_v4['weight'] = physio_df_v4.apply(assign_weight_2, axis = 1)
physio_df_v4 = physio_df_v4.drop(columns = ['Weight', 'patientweight'])

In [ ]:
### Deal with missing values in patient weight
weight_missing_value = physio_df_v4[physio_df_v4['weight'].isna()]
physio_df_v4 = physio_df_v4[~physio_df_v4['subject_id'].isin(pd.unique(weight_missing_value['subject_id']))]

In [ ]:
physio_df_v4.info()

In [ ]:
physio_df_v4 = physio_df_v4.reset_index(drop = True)

In [ ]:
# physio_df_v4.to_csv('physio_df_v4.csv', index = False)

In [ ]:
columns_with_missing_values = physio_df_v4.columns[physio_df_v4.isnull().any()].tolist()
physio_df_v4_pre = physio_df_v4[columns_with_missing_values].copy()

In [ ]:
columns_with_missing_values

In [ ]:
def process_chunk(chunk, imputer):
    
    chunk_imputed = imputer.fit_transform(chunk)  
    
    return chunk_imputed

In [ ]:
physio_df_v4.info()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
physio_df_v4_pre[columns_with_missing_values] = scaler.fit_transform(physio_df_v4_pre[columns_with_missing_values])

In [ ]:
len(physio_df_v4_pre)

In [ ]:
chunk_size = 10000  
chunks = [physio_df_v4_pre.iloc[i:i + chunk_size] for i in range(0, len(physio_df_v4_pre), chunk_size)]

In [ ]:
results = Parallel(n_jobs = 60)(
    delayed(process_chunk)(chunk, imputer) 
    for chunk in tqdm(chunks, desc = "KNN Imputation Progress")
)

In [ ]:
physio_df_v4_pre = pd.concat(
    [pd.DataFrame(result, columns = columns_with_missing_values) for result in results],
    ignore_index = True
)

In [ ]:
physio_df_v4_pre

In [ ]:
physio_df_v4_pre[columns_with_missing_values] = scaler.inverse_transform(physio_df_v4_pre[columns_with_missing_values])

In [ ]:
columns_with_missing_values

In [ ]:
physio_df_v4 = physio_df_v4.reset_index(drop = True)

In [ ]:
physio_df_v4.info()

In [ ]:
physio_df_v4[columns_with_missing_values] = physio_df_v4_pre[columns_with_missing_values]

In [ ]:
# scaler = MinMaxScaler()

# with threadpool_limits(limits = 100):
#     physio_df_v4_pre[columns_with_missing_values] = scaler.fit_transform(physio_df_v4_pre[columns_with_missing_values])
    
#     physio_df_v4_pre[columns_with_missing_values] = imputer.fit_transform(physio_df_v4_pre[columns_with_missing_values])
    
#     physio_df_v4_pre[columns_with_missing_values] = scaler.inverse_transform(physio_df_v4_pre[columns_with_missing_values])

# physio_df_v4[columns_with_missing_values] = physio_df_v4_pre[columns_with_missing_values]

In [ ]:
# scaler = MinMaxScaler()

# columns_with_missing_values = physio_df_v3.columns[
#     physio_df_v3.isnull().any()
# ].tolist()

# X = physio_df_v3[columns_with_missing_values].values  
# # X.shape = (n_rows, n_cols_with_nan)

# X_scaled = scaler.fit_transform(X)      
# X_imputed = imputer.fit_transform(X_scaled) 
# X_restored = scaler.inverse_transform(X_imputed) 

# physio_df_v3.loc[:, columns_with_missing_values] = X_restored

In [ ]:
physio_df_v4_pre.info()

In [ ]:
physio_df_v4.info()

In [ ]:
Counter(physio_df_v4[physio_df_v4['subject_id'] == 16133115]['readmission_count'])

In [ ]:
physio_df_v4[physio_df_v4['subject_id'] == 16133115][['subject_id', 'stay_id', 'discharge_fail', 'readmission', 'readmission_count']].iloc[60:100]

In [ ]:
physio_df_v4.to_csv('../icu_data/mimic_iv/physio_df_v4.csv', index = False)